In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from google.colab import drive

In [76]:
drive.mount('/content/drive')
df = pd.read_csv('drive/My Drive/Dataset/fake_news_train.csv')
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [77]:
# Drop nan values
df = df.dropna()

In [78]:
X = df.drop('label', axis = 1)

In [79]:
y = df['label']

In [80]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [81]:
X.shape

(18285, 4)

In [82]:
y.shape

(18285,)

In [83]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [84]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [85]:
voc_size = 5000

In [86]:
messages = X.copy()

In [87]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [88]:
messages.reset_index(inplace = True)

In [89]:
import nltk
import re
from nltk.corpus import stopwords

In [90]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [91]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [92]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [93]:
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  206 3061 1214]
 [   0    0    0 ... 1507 3621 2628]
 [   0    0    0 ... 1379 1830 4184]
 ...
 [   0    0    0 ... 3049 4402 4967]
 [   0    0    0 ... 3331  453 3293]
 [   0    0    0 ... 3696 2442 3141]]


In [94]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3890,
       2588,  469, 2381,  779, 2609,   76,  206, 3061, 1214], dtype=int32)

In [95]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss ='binary_crossentropy', optimizer='adam',metrics= ['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [96]:
embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model1.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [97]:
len(embedded_docs),y.shape

(18285, (18285,))

In [98]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [99]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final, test_size = 0.33, random_state = 42)

In [101]:
model1.fit(X_train,y_train,validation_data = (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 60ms/step - loss: 0.3010 - accuracy: 0.8513 - val_loss: 0.1956 - val_accuracy: 0.9186
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1360 - accuracy: 0.9482 - val_loss: 0.1923 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0972 - accuracy: 0.9643 - val_loss: 0.2411 - val_accuracy: 0.9160
Epoch 4/10
192/192 [==============================] - 11s 57ms/step - loss: 0.0670 - accuracy: 0.9771 - val_loss: 0.2817 - val_accuracy: 0.9100
Epoch 5/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0446 - accuracy: 0.9859 - val_loss: 0.3393 - val_accuracy: 0.9107
Epoch 6/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0346 - accuracy: 0.9891 - val_loss: 0.3695 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0187 - accuracy: 0.9944 - val_loss: 0.4590 - val_accuracy:

In [102]:
y_pred1 = (model1.predict(X_test) > 0.5 ).astype('int32')

In [103]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3074,  345],
       [ 171, 2445]])

In [104]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3419
           1       0.88      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.91      0.91      6035

